## 自注意力机制的实现

1. X * X^T 用于计算向量间的相关度，结果为对角阵；
2. self_attention，将输入X通过线性变换得到对应Q，K，V；
3. 基于Q，K得到自注意力权值，再通过softmax实现归一化权值mm；
4. 对V进行加权求和。

参考：https://mp.weixin.qq.com/s/G_sU1c3UGfHRZ2FVCT6Ucw

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, LayerNormalization, Dropout, Embedding
import numpy as np

输入矩阵
- 早：[1,2,1,2,1],
- 上：[1,1,3,2,1],
- 好：[3,1,2,1,1.0]

In [2]:
x=tf.constant([[1,2,1,2,1],[1,1,3,2,1],[3,1,2,1,1.0]])
x

<tf.Tensor: shape=(3, 5), dtype=float32, numpy=
array([[1., 2., 1., 2., 1.],
       [1., 1., 3., 2., 1.],
       [3., 1., 2., 1., 1.]], dtype=float32)>

In [3]:
tf.transpose(x)

<tf.Tensor: shape=(5, 3), dtype=float32, numpy=
array([[1., 1., 3.],
       [2., 1., 1.],
       [1., 3., 2.],
       [2., 2., 1.],
       [1., 1., 1.]], dtype=float32)>

矩阵`M`是一个方阵，，里面保存了每个向量，与自己和其他向量进行内积运算的结果。

In [4]:
m = tf.matmul(x,tf.transpose(x))
m

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[11., 11., 10.],
       [11., 16., 13.],
       [10., 13., 16.]], dtype=float32)>

对角阵，任意两个点积表征两个单位向量的夹角，一个向量在另一个向量上的投影。投影值越大，说明向量相关高；意味着在关注“早”的时候，更多的也会关注到“上”。

## 归一化

In [ ]:
mm = tf.nn.softmax(m)
mm

1. 在新的向量中，每个维度的数值均是其他词向量在对应维度上的加权求和，这个新的行向量就是"早"字的词向量经过注意力机制加权求和之后的表示。
2. 每一个字的向量都是，对所有向量进行加权求和；最终每个字都融合了自身和其他字的info

In [ ]:
tf.matmul(mm, x)

## self-attention
未直接使用seq_emb作为（X），进行运算，而要对其（X）进行线性变换得到Q，K，V再进行selft-attention

1. 为了提升模型的拟合能力；
2. `W_q，W_k，W_v`变换矩阵均是可训练，起到一个缓冲的效果。

1. 假设Q，K服从均值为0，方差为1的分布；，那么A=Q^T* K的分布为均值为0，方差为d；
2. 当d变得很大时，A的方差跟着变大，导致`mm`的分布会趋于陡峭(分布的方差大，分布集中在绝对值大的区域)。

归一化后的分布与d有关，处理后方差又变为1，从而使得训练过程中梯度值保持稳定。
